# Tensorflow Object Detection API and AWS Sagemaker

In this notebook, you will train and evaluate different models using the [Tensorflow Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/) and [AWS Sagemaker](https://aws.amazon.com/sagemaker/). 

If you ever feel stuck, you can refer to this [tutorial](https://aws.amazon.com/blogs/machine-learning/training-and-deploying-models-using-tensorflow-2-with-the-object-detection-api-on-amazon-sagemaker/).

## Dataset

We are using the [Waymo Open Dataset](https://waymo.com/open/) for this project. The dataset has already been exported using the tfrecords format. The files have been created following the format described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). You can find data stored on [AWS S3](https://aws.amazon.com/s3/), AWS Object Storage. The images are saved with a resolution of 640x640.

In [1]:
%%capture
%pip install tensorflow_io sagemaker -U

In [2]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework

Save the IAM role in a variable called `role`. This would be useful when training the model.

In [3]:
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::476375380884:role/service-role/AmazonSageMaker-ExecutionRole-20230227T161354


In [4]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
        'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://object-detection-project-jckuri/logs/'

## Container

To train the model, you will first need to build a [docker](https://www.docker.com/) container with all the dependencies required by the TF Object Detection API. The code below does the following:
* clone the Tensorflow models repository
* get the exporter and training scripts from the the repository
* build the docker image and push it 
* print the container name

In [5]:
%%bash

# clone the repo and get the scripts
git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Object Detection GitHub repository
cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
cp docker/models/research/object_detection/model_main_tf2.py source_dir

fatal: destination path 'docker/models' already exists and is not an empty directory.


In [6]:
# build and push the docker image. This code can be commented after being ran once.
# This will take around 10 mins.
image_name = 'tf2-object-detection'
!sh ./docker/build_and_push.sh $image_name

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Building image with name tf2-object-detection
Sending build context to Docker daemon  722.9MB
Step 1/17 : FROM tensorflow/tensorflow:2.9.0-gpu
 ---> c8d9ee2a0ff4
Step 2/17 : ARG DEBIAN_FRONTEND=noninteractive
 ---> Running in 2d0766a5e9a5
Removing intermediate container 2d0766a5e9a5
 ---> 85149043e2a4
Step 3/17 : RUN rm /etc/apt/sources.list.d/cuda.list
 ---> Running in 3ccda5590c3d
Removing intermediate container 3ccda5590c3d
 ---> 4a944b52f041
Step 4/17 : RUN apt-key del 7fa2af80
 ---> Running in 6b14f55f951c
OK
Removing intermediate container 6b14f55f951c
 ---> 620e97f10f40
Step 5/17 : RUN apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/3bf863cc.pub
 ---> Running in f76b1c0121a3
Executing: /tmp

Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gpg-wks-server amd64 2.2.19-3ubuntu2.2 [90.2 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gnupg-utils amd64 2.2.19-3ubuntu2.2 [481 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gpg-agent amd64 2.2.19-3ubuntu2.2 [232 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gpg amd64 2.2.19-3ubuntu2.2 [482 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gpgconf amd64 2.2.19-3ubuntu2.2 [124 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gnupg-l10n all 2.2.19-3ubuntu2.2 [51.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gnupg all 2.2.19-3ubuntu2.2 [259 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gpgsm amd64 2.2.19-3ubuntu2.2 [217 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 gpgv amd64 2.2.19-3ubuntu2.2 [200 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-upda

Get:89 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libx11-dev amd64 2:1.6.9-2ubuntu1.2 [647 kB]
Get:90 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libglx-dev amd64 1.3.2-1~ubuntu0.20.04.2 [14.0 kB]
Get:91 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgl-dev amd64 1.3.2-1~ubuntu0.20.04.2 [97.8 kB]
Get:92 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libegl-dev amd64 1.3.2-1~ubuntu0.20.04.2 [17.2 kB]
Get:93 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libjbig0 amd64 2.1-3.1ubuntu0.20.04.1 [27.3 kB]
Get:94 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libwebp6 amd64 0.6.1-2ubuntu0.20.04.1 [185 kB]
Get:95 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libtiff5 amd64 4.1.0+git191117-2ubuntu0.20.04.8 [163 kB]
Get:96 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgdk-pixbuf2.0-common all 2.40.0+dfsg-3ubuntu0.4 [4592 B]
Get:97 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgdk

Selecting previously unselected package libxau6:amd64.
Preparing to unpack .../010-libxau6_1%3a1.0.9-0ubuntu1_amd64.deb ...
Unpacking libxau6:amd64 (1:1.0.9-0ubuntu1) ...
Selecting previously unselected package libxdmcp6:amd64.
Preparing to unpack .../011-libxdmcp6_1%3a1.1.3-0ubuntu1_amd64.deb ...
Unpacking libxdmcp6:amd64 (1:1.1.3-0ubuntu1) ...
Selecting previously unselected package libxcb1:amd64.
Preparing to unpack .../012-libxcb1_1.14-2_amd64.deb ...
Unpacking libxcb1:amd64 (1.14-2) ...
Selecting previously unselected package libx11-data.
Preparing to unpack .../013-libx11-data_2%3a1.6.9-2ubuntu1.2_all.deb ...
Unpacking libx11-data (2:1.6.9-2ubuntu1.2) ...
Selecting previously unselected package libx11-6:amd64.
Preparing to unpack .../014-libx11-6_2%3a1.6.9-2ubuntu1.2_amd64.deb ...
Unpacking libx11-6:amd64 (2:1.6.9-2ubuntu1.2) ...
Selecting previously unselected package libxext6:amd64.
Preparing to unpack .../015-libxext6_2%3a1.3.4-0ubuntu1_amd64.deb ...
Unpacking libxext6:amd64 (

Selecting previously unselected package libegl1:amd64.
Preparing to unpack .../057-libegl1_1.3.2-1~ubuntu0.20.04.2_amd64.deb ...
Unpacking libegl1:amd64 (1.3.2-1~ubuntu0.20.04.2) ...
Selecting previously unselected package libxcb-glx0:amd64.
Preparing to unpack .../058-libxcb-glx0_1.14-2_amd64.deb ...
Unpacking libxcb-glx0:amd64 (1.14-2) ...
Selecting previously unselected package libxfixes3:amd64.
Preparing to unpack .../059-libxfixes3_1%3a5.0.3-2_amd64.deb ...
Unpacking libxfixes3:amd64 (1:5.0.3-2) ...
Selecting previously unselected package libxxf86vm1:amd64.
Preparing to unpack .../060-libxxf86vm1_1%3a1.1.4-1build1_amd64.deb ...
Unpacking libxxf86vm1:amd64 (1:1.1.4-1build1) ...
Selecting previously unselected package libllvm12:amd64.
Preparing to unpack .../061-libllvm12_1%3a12.0.0-3ubuntu1~20.04.5_amd64.deb ...
Unpacking libllvm12:amd64 (1:12.0.0-3ubuntu1~20.04.5) ...
Selecting previously unselected package libsensors-config.
Preparing to unpack .../062-libsensors-config_1%3a3.6.0

Selecting previously unselected package python3-soupsieve.
Preparing to unpack .../102-python3-soupsieve_1.9.5+dfsg-1_all.deb ...
Unpacking python3-soupsieve (1.9.5+dfsg-1) ...
Selecting previously unselected package python3-bs4.
Preparing to unpack .../103-python3-bs4_4.8.2-1_all.deb ...
Unpacking python3-bs4 (4.8.2-1) ...
Selecting previously unselected package python3-ply.
Preparing to unpack .../104-python3-ply_3.11-3ubuntu0.1_all.deb ...
Unpacking python3-ply (3.11-3ubuntu0.1) ...
Selecting previously unselected package python3-pycparser.
Preparing to unpack .../105-python3-pycparser_2.19-1ubuntu1_all.deb ...
Unpacking python3-pycparser (2.19-1ubuntu1) ...
Selecting previously unselected package python3-cffi.
Preparing to unpack .../106-python3-cffi_1.14.0-1build1_all.deb ...
Unpacking python3-cffi (1.14.0-1build1) ...
Selecting previously unselected package python3-xcffib.
Preparing to unpack .../107-python3-xcffib_0.8.1-0.8_amd64.deb ...
Unpacking python3-xcffib (0.8.1-0.8) ...


Setting up libdrm-intel1:amd64 (2.4.107-8ubuntu1~20.04.2) ...
Setting up libgl1-mesa-dri:amd64 (21.2.6-0ubuntu0.1~20.04.2) ...
Setting up libx11-dev:amd64 (2:1.6.9-2ubuntu1.2) ...
Setting up libxext6:amd64 (2:1.3.4-0ubuntu1) ...
Setting up libcairo2:amd64 (1.16.0-4ubuntu1) ...
Setting up libxxf86vm1:amd64 (1:1.1.4-1build1) ...
Setting up libegl-mesa0:amd64 (21.2.6-0ubuntu0.1~20.04.2) ...
Setting up libxfixes3:amd64 (1:5.0.3-2) ...
Setting up libgdk-pixbuf2.0-0:amd64 (2.40.0+dfsg-3ubuntu0.4) ...
Setting up python3-cairocffi (0.9.0-4) ...
Setting up xauth (1:1.1-0ubuntu1) ...
Setting up libgdk-pixbuf2.0-bin (2.40.0+dfsg-3ubuntu0.4) ...
Setting up libegl1:amd64 (1.3.2-1~ubuntu0.20.04.2) ...
Setting up gnupg (2.2.19-3ubuntu2.2) ...
Setting up libxss1:amd64 (1:1.2.3-1) ...
Setting up libxft2:amd64 (2.3.3-0ubuntu1) ...
Setting up libglx-mesa0:amd64 (21.2.6-0ubuntu0.1~20.04.2) ...
Setting up libglx0:amd64 (1.3.2-1~ubuntu0.20.04.2) ...
Setting up libtk8.6:amd64 (8.6.10-1) ...
Setting up libgl1

To verify that the image was correctly pushed to the [Elastic Container Registry](https://aws.amazon.com/ecr/), you can look at it in the AWS webapp. For example, below you can see that three different images have been pushed to ECR. You should only see one, called `tf2-object-detection`.
![ECR Example](../data/example_ecr.png)


In [7]:
# display the container name
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

476375380884.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection:20230324195431


## Pre-trained model from model zoo

As often, we are not training from scratch and we will be using a pretrained model from the TF Object Detection model zoo. You can find pretrained checkpoints [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Because your time is limited for this project, we recommend to only experiment with the following models:
* SSD MobileNet V2 FPNLite 640x640	
* SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	
* Faster R-CNN ResNet50 V1 640x640	
* EfficientDet D1 640x640	
* Faster R-CNN ResNet152 V1 640x640	

In the code below, the EfficientDet D1 model is downloaded and extracted. This code should be ajusted if you were to experiment with other architectures.

In [8]:
%%bash
mkdir /tmp/checkpoint
mkdir source_dir/checkpoint

#wget -O /tmp/efficientdet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
#wget -O /tmp/ssd_mobilenet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
wget -O /tmp/retinanet50.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

#tar -zxvf /tmp/efficientdet.tar.gz --strip-components 2 --directory source_dir/checkpoint efficientdet_d1_coco17_tpu-32/checkpoint
#tar -zxvf /tmp/ssd_mobilenet.tar.gz --strip-components 2 --directory source_dir/checkpoint ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint
tar -zxvf /tmp/retinanet50.tar.gz --strip-components 2 --directory source_dir/checkpoint ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint

#tar -zxvf /tmp/ssd_mobilenet.tar.gz --strip-components 2 --directory . ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/pipeline.config


ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.index


mkdir: cannot create directory ‘/tmp/checkpoint’: File exists
mkdir: cannot create directory ‘source_dir/checkpoint’: File exists
--2023-03-24 19:55:28--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.253.63.128, 2607:f8b0:4004:c0b::80
Connecting to download.tensorflow.org (download.tensorflow.org)|172.253.63.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244817203 (233M) [application/x-tar]
Saving to: ‘/tmp/retinanet50.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 14.0M 17s
    50K .......... .......... .......... .......... ..........  0% 24.9M 13s
   100K .......... .......... .......... .......... ..........  0% 26.2M 12s
   150K .......... .......... .......... .......... ..........  0% 27.4M 11s
   200K .......... .......... .......... .......... ..........  0% 27.1M 10s
   250K .......

## Edit pipeline.config file

The [`pipeline.config`](source_dir/pipeline.config) in the `source_dir` folder should be updated when you experiment with different models. The different config files are available [here](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2).

>Note: The provided `pipeline.config` file works well with the `EfficientDet` model. You would need to modify it when working with other models.

## Launch Training Job

Now that we have a dataset, a docker image and some pretrained model weights, we can launch the training job. To do so, we create a [Sagemaker Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html), where we indicate the container name, name of the config file, number of training steps etc.

The `run_training.sh` script does the following:
* train the model for `num_train_steps` 
* evaluate over the val dataset
* export the model

Different metrics will be displayed during the evaluation phase, including the mean average precision. These metrics can be used to quantify your model performances and compare over the different iterations.

You can also monitor the training progress by navigating to **Training -> Training Jobs** from the Amazon Sagemaker dashboard in the Web UI.

In [9]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point='run_training.sh',
    source_dir='source_dir/',
    hyperparameters={
        "model_dir":"/opt/training",        
        "pipeline_config_path": "pipeline.config",
        "num_train_steps": "10000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='tf2-object-detection'
)

estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: tf2-object-detection-2023-03-24-19-55-33-029


2023-03-24 19:55:44 Starting - Starting the training job...
2023-03-24 19:56:20 Starting - Preparing the instances for training.........
2023-03-24 19:57:47 Downloading - Downloading input data
2023-03-24 19:57:47 Training - Downloading the training image...............
2023-03-24 20:00:13 Training - Training image download completed. Training in progress....2023-03-24 20:00:42,167 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-03-24 20:00:42,200 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-03-24 20:00:42,234 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-03-24 20:00:42,247 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train",
        "val": "/opt/ml/input/data/val"
    },
    "current_host": "algo-1",
    "curre

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
W0324 20:00:57.742027 140103179310912 deprecation.py:350] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
W0324 20:01:00.486734 140103179310912 deprecation.py:554] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: calling reduce_sum_v1 (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use `tf.cast` instead.
W0324 20:01:01.200707 140103179310912 deprecation.py:350] From /usr/local/lib/p

INFO:tensorflow:Step 400 per-step time 0.383s
I0324 20:04:56.979272 140103179310912 model_lib_v2.py:705] Step 400 per-step time 0.383s
INFO:tensorflow:{'Loss/classification_loss': 0.24852116,
 'Loss/localization_loss': 0.32642815,
 'Loss/regularization_loss': 0.24150418,
 'Loss/total_loss': 0.81645346,
 'learning_rate': 0.0005}
I0324 20:04:56.979575 140103179310912 model_lib_v2.py:708] {'Loss/classification_loss': 0.24852116,
 'Loss/localization_loss': 0.32642815,
 'Loss/regularization_loss': 0.24150418,
 'Loss/total_loss': 0.81645346,
 'learning_rate': 0.0005}
INFO:tensorflow:Step 500 per-step time 0.383s
I0324 20:05:35.284716 140103179310912 model_lib_v2.py:705] Step 500 per-step time 0.383s
INFO:tensorflow:{'Loss/classification_loss': 0.31672373,
 'Loss/localization_loss': 0.4233984,
 'Loss/regularization_loss': 0.23117891,
 'Loss/total_loss': 0.9713011,
 'learning_rate': 0.0005}
I0324 20:05:35.285062 140103179310912 model_lib_v2.py:708] {'Loss/classification_loss': 0.31672373,
 'Lo

INFO:tensorflow:Step 1800 per-step time 0.383s
I0324 20:13:57.019853 140103179310912 model_lib_v2.py:705] Step 1800 per-step time 0.383s
INFO:tensorflow:{'Loss/classification_loss': 0.23443054,
 'Loss/localization_loss': 0.2957376,
 'Loss/regularization_loss': 0.21322832,
 'Loss/total_loss': 0.7433964,
 'learning_rate': 0.0005}
I0324 20:13:57.020232 140103179310912 model_lib_v2.py:708] {'Loss/classification_loss': 0.23443054,
 'Loss/localization_loss': 0.2957376,
 'Loss/regularization_loss': 0.21322832,
 'Loss/total_loss': 0.7433964,
 'learning_rate': 0.0005}
INFO:tensorflow:Step 1900 per-step time 0.383s
I0324 20:14:35.353732 140103179310912 model_lib_v2.py:705] Step 1900 per-step time 0.383s
INFO:tensorflow:{'Loss/classification_loss': 0.17426692,
 'Loss/localization_loss': 0.20979032,
 'Loss/regularization_loss': 0.21283323,
 'Loss/total_loss': 0.59689045,
 'learning_rate': 0.0005}
I0324 20:14:35.354021 140103179310912 model_lib_v2.py:708] {'Loss/classification_loss': 0.17426692,
 '

...

INFO:tensorflow:Step 6000 per-step time 0.385s
I0324 20:41:00.023887 140103179310912 model_lib_v2.py:705] Step 6000 per-step time 0.385s
INFO:tensorflow:{'Loss/classification_loss': 0.07489116,
 'Loss/localization_loss': 0.10545574,
 'Loss/regularization_loss': 0.12903453,
 'Loss/total_loss': 0.30938143,
 'learning_rate': 5e-05}
I0324 20:41:00.024255 140103179310912 model_lib_v2.py:708] {'Loss/classification_loss': 0.07489116,
 'Loss/localization_loss': 0.10545574,
 'Loss/regularization_loss': 0.12903453,
 'Loss/total_loss': 0.30938143,
 'learning_rate': 5e-05}
INFO:tensorflow:Step 6100 per-step time 0.414s
I0324 20:41:41.377555 140103179310912 model_lib_v2.py:705] Step 6100 per-step time 0.414s
INFO:tensorflow:{'Loss/classification_loss': 0.07653737,
 'Loss/localization_loss': 0.06389807,
 'Loss/regularization_loss': 0.12826478,
 'Loss/total_loss': 0.26870024,
 'learning_rate': 5e-05}
I0324 20:41:41.377870 140103179310912 model_lib_v2.py:708] {'Loss/classification_loss': 0.07653737,
 

INFO:tensorflow:Step 7400 per-step time 0.383s
I0324 20:50:03.645691 140103179310912 model_lib_v2.py:705] Step 7400 per-step time 0.383s
INFO:tensorflow:{'Loss/classification_loss': 0.05567722,
 'Loss/localization_loss': 0.053051777,
 'Loss/regularization_loss': 0.12059753,
 'Loss/total_loss': 0.22932652,
 'learning_rate': 5e-05}
I0324 20:50:03.646024 140103179310912 model_lib_v2.py:708] {'Loss/classification_loss': 0.05567722,
 'Loss/localization_loss': 0.053051777,
 'Loss/regularization_loss': 0.12059753,
 'Loss/total_loss': 0.22932652,
 'learning_rate': 5e-05}
INFO:tensorflow:Step 7500 per-step time 0.384s
I0324 20:50:42.005855 140103179310912 model_lib_v2.py:705] Step 7500 per-step time 0.384s
INFO:tensorflow:{'Loss/classification_loss': 0.06044603,
 'Loss/localization_loss': 0.071165286,
 'Loss/regularization_loss': 0.1200839,
 'Loss/total_loss': 0.25169522,
 'learning_rate': 1e-05}
I0324 20:50:42.006161 140103179310912 model_lib_v2.py:708] {'Loss/classification_loss': 0.06044603,

INFO:tensorflow:Step 8800 per-step time 0.385s
I0324 20:59:03.749959 140103179310912 model_lib_v2.py:705] Step 8800 per-step time 0.385s
INFO:tensorflow:{'Loss/classification_loss': 0.05394963,
 'Loss/localization_loss': 0.071170956,
 'Loss/regularization_loss': 0.11688744,
 'Loss/total_loss': 0.24200803,
 'learning_rate': 1e-05}
I0324 20:59:03.750286 140103179310912 model_lib_v2.py:708] {'Loss/classification_loss': 0.05394963,
 'Loss/localization_loss': 0.071170956,
 'Loss/regularization_loss': 0.11688744,
 'Loss/total_loss': 0.24200803,
 'learning_rate': 1e-05}
INFO:tensorflow:Step 8900 per-step time 0.383s
I0324 20:59:42.073919 140103179310912 model_lib_v2.py:705] Step 8900 per-step time 0.383s
INFO:tensorflow:{'Loss/classification_loss': 0.06456618,
 'Loss/localization_loss': 0.06863957,
 'Loss/regularization_loss': 0.11665967,
 'Loss/total_loss': 0.24986541,
 'learning_rate': 1e-05}
I0324 20:59:42.074210 140103179310912 model_lib_v2.py:708] {'Loss/classification_loss': 0.06456618,

Instructions for updating:
Use `tf.cast` instead.
W0324 21:07:03.464618 140413280651072 deprecation.py:350] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Waiting for new checkpoint at /opt/training
I0324 21:07:06.128785 140413280651072 checkpoint_utils.py:140] Waiting for new checkpoint at /opt/training
INFO:tensorflow:Found new checkpoint at /opt/training/ckpt-11
I0324 21:07:06.129632 140413280651072 checkpoint_utils.py:149] Found new checkpoint at /opt/training/ckpt-11
/usr/local/lib/python3.8/dist-packages/keras/backend.py:451: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(
Instructions fo

W0324 21:08:36.480422 140035595700032 save_impl.py:66] Skipping full serialization of Keras layer <object_detection.meta_architectures.ssd_meta_arch.SSDMetaArch object at 0x7f5bdec77c10>, because it is not built.
W0324 21:08:58.293447 140035595700032 save.py:271] Found untraced functions such as WeightSharedConvolutionalBoxPredictor_layer_call_fn, WeightSharedConvolutionalBoxPredictor_layer_call_and_return_conditional_losses, WeightSharedConvolutionalBoxHead_layer_call_fn, WeightSharedConvolutionalBoxHead_layer_call_and_return_conditional_losses, WeightSharedConvolutionalClassHead_layer_call_fn while saving (showing 5 of 278). These functions will not be directly callable after loading.
INFO:tensorflow:Assets written to: /tmp/exported/saved_model/assets
I0324 21:09:06.091465 140035595700032 builder_impl.py:797] Assets written to: /tmp/exported/saved_model/assets
INFO:tensorflow:Writing pipeline config file to /tmp/exported/pipeline.config
I0324 21:09:07.630016 140035595700032 config_ut

You should be able to see your model training in the AWS webapp as shown below:
![ECR Example](../data/example_trainings.png)


In [10]:
import sagemaker
sagemaker_session = sagemaker.Session()
aws_region = sagemaker_session.boto_region_name
print('aws_region={}'.format(aws_region))

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


aws_region=us-east-1


In [11]:
job_artifacts_path = estimator.latest_job_tensorboard_artifacts_path()

!echo "pip install 'tensorflow<2.4'"
!echo "pip install 'tensorflow-io<2.4'"
!echo "pip install 'tensorboard<2.4'"
!echo "AWS_REGION={aws_region}"
!echo "tensorboard --logdir={job_artifacts_path}"

pip install 'tensorflow<2.4'
pip install 'tensorflow-io<2.4'
pip install 'tensorboard<2.4'
AWS_REGION=us-east-1
tensorboard --logdir=s3://object-detection-project-jckuri/logs/tf2-object-detection-2023-03-24-19-55-33-029/tensorboard-output


In [12]:
class URL:
    
    def __init__(self, url):
        self.url = url
    
    def _repr_html_(self):
        return '<a href="{}">{}</a>'.format(self.url, self.url)

In [13]:
# https://<notebook instance hostname>/proxy/6006/
jupyter_notebook_url = 'https://object-detection-project-a5lc.notebook.us-east-1.sagemaker.aws'
url = '{}/proxy/6006/'.format(jupyter_notebook_url)

o = URL(url)
o

## Improve on the initial model

Most likely, this initial experiment did not yield optimal results. However, you can make multiple changes to the `pipeline.config` file to improve this model. One obvious change consists in improving the data augmentation strategy. The [`preprocessor.proto`](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto) file contains the different data augmentation method available in the Tf Object Detection API. Justify your choices of augmentations in the writeup.

Keep in mind that the following are also available:
* experiment with the optimizer: type of optimizer, learning rate, scheduler etc
* experiment with the architecture. The Tf Object Detection API model zoo offers many architectures. Keep in mind that the pipeline.config file is unique for each architecture and you will have to edit it.
* visualize results on the test frames using the `2_deploy_model` notebook available in this repository.

In the cell below, write down all the different approaches you have experimented with, why you have chosen them and what you would have done if you had more time and resources. Justify your choices using the tensorboard visualizations (take screenshots and insert them in your writeup), the metrics on the evaluation set and the generated animation you have created with [this tool](../2_run_inference/2_deploy_model.ipynb).

In [14]:
# your writeup goes here.

**I preferred to write all my documentation in the file <big><big>[README.md](../README.md)</big></big>.<br/>
Please, click on this file and read it because it explains all the items in the rubric.**